**Helper functions**

In [ ]:
import numpy as np
import cvxpy as cp
import itertools
from tqdm.auto import tqdm
import math
from functools import cache

def min_norm_convex_combination_weights(vectors, lowerbound=None):
    """
    Finds the weights of the "convex" (but with possibly negative weights)
    combination of the given vectors with minimal norm.

    Args:
        vectors: A list of numpy arrays representing the vectors.
        lowerbound: float; force weights to be >= lowerbound.

    Returns:
        A numpy array representing the weights of the "convex" combination with minimal norm.
    """

    n = len(vectors)  # Number of vectors
    x = cp.Variable(n)  # Define the optimization variable

    # Define the objective function to minimize the norm of the convex combination
    objective = cp.Minimize(cp.norm(cp.sum([x[i] * vectors[i] for i in range(n)], axis=0)))

    # Define constraints: weights must sum to 1, and optionally be >= lowerbound
    if lowerbound is None:
        constraints = [cp.sum(x) == 1]  # Weights must sum to 1 but can be negative
    else:
        constraints = [cp.sum(x) == 1, x >= lowerbound]  # Weights must sum to 1 and be >= lowerbound

    prob = cp.Problem(objective, constraints)  # Define the optimization problem
    prob.solve()  # Solve the problem

    return x.value  # Return the optimal weights

def row_permutations(matrix):
    """
    Lists all row permutations of a given matrix.

    Args:
        matrix: The input matrix.

    Returns:
        A list of all row permutations of the matrix.
    """

    result = []
    # Generate all permutations of row indices
    for permutation in itertools.permutations(range(matrix.shape[0])):
        permuted_matrix = matrix[permutation, :]  # Apply permutation to rows
        result.append(permuted_matrix)  # Add to result list
    return result

@cache #compute only once
def list_of_all_permutation_matrices(n):
    """
    Returns a list of all n x n permutation matrices.

    Args:
        n: Dimension of matrix

    Returns:
        A list of all n x n permutation matrices.
    """

    permutations = row_permutations(np.identity(n))  # Generate permutation matrices from identity matrix
    return permutations

def permutation_matrices_dom(matrix_A):
    """
    Outputs all (flattened) nxn permutation matrices entrywise dominated by A.

    Args:
        matrix_A: n x n matrix (not flattened)

    Returns:
        A list of all n x n permutation matrices entrywise dominated by A, flattened to vectors.
    """

    n = matrix_A.shape[0]  # Matrix dimension
    matrices = list_of_all_permutation_matrices(n)  # Get all permutation matrices
    result = []
    # Check if each permutation matrix is entrywise dominated by matrix_A
    for matrix in matrices:
        if np.all(matrix <= matrix_A):
            result.append(matrix.flatten())  # Flatten matrix and add to result
    return result

def check_weights_nonneg(A):
    """
    Checks if the norm-minimizing weights are nonnegative.

    Args:
        A: n x n matrix

    Returns:
        True if all weights are nonnegative, False otherwise.
    """

    weights = min_norm_convex_combination_weights(vectors=permutation_matrices_dom(A))
    return np.all(weights >= 0)  # Check if all weights are nonnegative

def latex_matrix(a):
    """
    Returns a LaTeX bmatrix.

    Args:
        a: numpy array

    Returns:
        LaTeX bmatrix as a string
    """

    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')

    lines = np.array2string(a, max_line_width=np.infty, precision=3).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv += [r'\end{bmatrix}']
    return '\n'.join(rv)



def per(mtx, column, selected, prod, output=False):
    """
    Helper for permanent() function. Recursively computes the permanent of a matrix using row expansion.

    Given an n-by-n matrix A, the permanent of an n-by-n matrix A is the sum of
    the products of its entries A[i, s[i]],
    where i ranges from 0 to n-1 and s is a permutation of the set {0, 1, ..., n-1}.
    This function computes the permanent by recursively expanding along columns.

    SOURCE: https://homepages.math.uic.edu/~jan/mcs507f13/permanent.py

    Args:
        mtx (np.ndarray): The input n-by-n matrix whose permanent is to be computed.
        column (int): The current column being processed.
        selected (list): A list of indices representing the rows that have been selected in the current permutation.
        prod (float): The accumulated product of selected matrix entries.
        output (bool, optional): If True, prints the current permutation and product. Default is False.

    Returns:
        float: The permanent of the matrix if output is False.
    """

    # Base case: if all columns have been processed, return the accumulated product
    if column == mtx.shape[1]:
        if output:
            print(selected, prod)
        return prod
    else:
        result = 0
        # Iterate over all rows to find valid permutations
        for row in range(mtx.shape[0]):
            if row not in selected:
                # Recursive call to process the next column with the current row added to the selection
                result += per(mtx, column + 1, selected + [row], prod * mtx[row, column])
        return result

def permanent(mat):
    """
    Computes the permanent of a matrix using the recursive row expansion method.

    This function initializes the recursive computation of the permanent by calling the helper function `per`
    with the initial parameters.

    SOURCE: https://homepages.math.uic.edu/~jan/mcs507f13/permanent.py

    Args:
        mat (np.ndarray): The input n-by-n matrix whose permanent is to be computed.

    Returns:
        float: The computed permanent of the matrix.
    """

    # Start the recursion with the first column, empty list of selected rows, and an initial product of 1
    return per(mat, 0, [], 1)


def check_if_face(matrices):
    """
    Given a list of permutation matrices, checks if they define a face of the Birkhoff polytope.

    Args:
        matrices: List of permutation matrices

    Returns:
        True if the matrices define a face of the Birkhoff polytope, False otherwise.
    """

    l = len(matrices)
    if l > 1:
        A = np.maximum.reduce(matrices)  # Element-wise maximum of the matrices
        p = permanent(A)  # Compute the permanent of matrix A
        return p == l  # Check if the permanent equals the number of matrices
    else:
        return True  # Single matrix always defines a face

def list_of_all_subsets(s):
    """
    Returns all subsets (as lists) of a given list.

    Args:
        s: List of elements

    Returns:
        A list of all subsets of s.
    """

    combs = []
    for i in range(len(s) + 1):
        els = [list(x) for x in itertools.combinations(s, i)]
        combs.extend(els)
    return combs

def has_alternative_positive_weights(weights, set_of_perm_matrices):
    """
    In some cases, weights close to zero are obtained by
    min_norm_convex_combination_weights() despite the argmin (minimum-norm point)
    being in the interior of the face. This function solves the minimization problem under
    a lower bound constraint on the weights and checks if we find the same argmin.
    If yes, we can conclude that we are in the aforementioned situation

    For n=4 there is essentially just one face where this applies. It has four vertices forming a
    2-dimensional rectangle. The argmin lies in the center. Hence it can be written
    as a convex combination of just two (diagonally opposite) vertices, thus two weights are zero.
    But it can also be written as a convex combination of all vertices, with
    weights 1/4, and that representation shows that the argmin is in the interior.

    Args:
        weights: Weights obtained from the minimization problem
        set_of_perm_matrices: List of permutation matrices

    Returns:
        bool: True if an alternative representation has all positive weights, False otherwise.
    """

    #get optimal weights under the constraint of all weights being >= 0.01
    alt_weights = min_norm_convex_combination_weights([p.flatten() for p in set_of_perm_matrices], lowerbound=0.01)

    #check if we find the same argmin (minimum-norm point) as without the lower bound
    argmin = 0
    alt_argmin = 0
    for i in range(len(set_of_perm_matrices)):
        argmin += weights[i] * set_of_perm_matrices[i]
        alt_argmin += alt_weights[i] * set_of_perm_matrices[i]

    return np.allclose(argmin, alt_argmin)

def check_if_equivalent_to_special_matrix(matrix):
    """
    Checks if an n x n binary matrix is equal, up to permutation of rows and columns,
    to a specific matrix with ones in the first row, first column, and diagonal, and zeros elsewhere.

    Args:
        matrix: The input n x n matrix as a NumPy array.

    Returns:
        bool: True if the matrix is equivalent to the special matrix, False otherwise.
    """

    n = matrix.shape[0]

    # Delete the first row of ones
    for i in range(n):
        if np.all(matrix[i, :] == 1):
            matrix = np.delete(matrix, i, axis=0)
            break

    # Delete the first column of ones
    for i in range(n):
        if np.all(matrix[:, i] == 1):
            matrix = np.delete(matrix, i, axis=1)
            break

    if not matrix.shape == (n - 1, n - 1):
        return False

    # Check if the remaining matrix equals the identity matrix up to permutation
    for i in range(n - 1):
        if np.sum(matrix[i, :]) != 1 or np.sum(matrix[:, i]) != 1:
            return False

    return True

**Verify that the Birkhoff polytope of n x n matrices for n=4 (or less) has the following property:
for each face F, the minimum-norm point of the affine hull of F lies in F**

In [ ]:
'''
This code is intended for marginal dimension n=4 (or less). It loops through all subsets of n x n permutation matrices,
checks if a subset defines a face F of the Birkhoff polytope, then checks if the minimum-norm point of
the affine hull of F lies within F. If the point lies well within the interior (weights are > 0.01), the face passes.
If the point numerically lies close to the boundary, the program checks that the face is equivalent
(up to relabeling the points in the marginals, i.e., simultaneously permuting
the rows and columns of the permutation matrices) to a particular face for
which it was shown analytically that the point lies exactly on the boundary. That
particular face is given by all permutation matrices entrywise doiminated by the
"special matrix" with ones in the first row, first column, and diagonal, and zeros elsewhere.
'''

n = 4  # Dimension of the matrix

# Driver code

subsets_of_perm_matrices = list_of_all_subsets(list_of_all_permutation_matrices(n))  # Get all subsets of n x n permutation matrices

bad_counter = 0
face_counter = 0
equiv_counter = 0

for set_of_perm_matrices in tqdm(subsets_of_perm_matrices):
    if len(set_of_perm_matrices) > 0 and check_if_face(set_of_perm_matrices): # check if the current set of permutation matrices defines a face
        face_counter += 1

        # Compute weights of minimum norm point of the affine hull of the face
        weights = min_norm_convex_combination_weights([p.flatten() for p in set_of_perm_matrices])

        # If weights are > 0.01, face is fine.
        # If not, check if there are other weghts for the minimum norm point of the affine hull that are > 0.01. If yes, face is also fine.
        if not np.all(weights > 0.01) and not has_alternative_positive_weights(weights, set_of_perm_matrices):

            # If not, check if the face is equivalent (up to symmetries) to the particular face studied analytically in the paper
            print(f'vertices = ')
            for p in set_of_perm_matrices:
                print(p)
            A = np.maximum.reduce(set_of_perm_matrices)
            print(f'max is A=\n{A}')
            equiv = check_if_equivalent_to_special_matrix(A)
            print(f'Equivalent? {equiv}')
            print(f'weights = {weights}\n')
            if equiv:
                equiv_counter += 1
            else:
                bad_counter += 1 # If none of the above "good" cases applies, increment bad_counter. Such "bad" cases, if any, require further checking. (There are none for n <= 4.)

print(f'number of nonempty faces: {face_counter}')
print(f'narrow cases equivalent to special matrix: {equiv_counter}')
print(f'narrow cases not equivalent to special matrix: {bad_counter}')

  0%|          | 0/16777216 [00:00<?, ?it/s]

vertices = 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
max is A=
[[1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [1. 1. 1. 1.]
 [0. 0. 1. 1.]]
Equivalent? True
weights = [-6.32577406e-17  3.33333333e-01  3.33333333e-01  3.33333333e-01]

vertices = 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]
max is A=
[[1. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 1. 1.]
 [1. 1. 1. 1.]]
Equivalent? True
weights = [3.33333333e-01 3.42819645e-17 3.33333333e-01 3.33333333e-01]

vertices = 
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 0. 0. 1.